In [ ]:
# 0. import packages

import sys
import string

import time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plot

In [ ]:
# #1.1 GPU stuff

# print ("cuda: ", torch.cuda.is_available())
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# print ("current device: ", device)
# print ("count: ", torch.cuda.device_count())

# if torch.cuda.is_available():
#     print ("device name: ", torch.cuda.get_device_name(0))
#     torch.cuda.set_device(0)

In [ ]:
# 1.2 load groove dataset
import math

groove_csv = pd.read_csv('groove/info.csv')
print("groove dataset:", len(groove_csv))

# get train, test, and validation sets
train_csv = []
test_csv = []
validation_csv = []

for index, row in groove_csv.iterrows():
    if str(row.audio_filename).lower() != "nan":
        split = row['split']
        if split == "train":
            train_csv.append(row)
        elif split == "test":
            test_csv.append(row)
        elif split == "validation":
            validation_csv.append(row)
        
print ("train: ", len(train_csv))
print ("test: ", len(test_csv))
print ("validation: ", len(validation_csv))

print (train_csv[0].midi_filename)

In [ ]:
# convert audio files into tensor
from pydub import AudioSegment
from scipy import signal
import librosa
import torch

def load_audio_to_melspec_tensor(wavpath, sample_rate=44_100):
    window_size = 0.025
    window_stride = 0.01
    n_dft = 2048
    win_length = int(sample_rate * window_size)
    hop_length = int(sample_rate * window_stride)
    y, sr = librosa.load(wavpath)
    y = y - y.mean()
    y = np.append(y[0],y[1:]-.97*y[:-1])
    # compute mel spectrogram
    stft = librosa.stft(y, n_fft=n_dft, hop_length=hop_length, win_length=win_length, window=signal.hamming)
    spec = np.abs(stft)**2
    mel_basis = librosa.filters.mel(sr=sample_rate, n_fft=n_dft, n_mels=40, fmin=20)
    melspec = np.dot(mel_basis, spec)

    logspec = librosa.power_to_db(melspec, ref=np.max)
    logspec = np.transpose(logspec)

    plot.imshow(logspec.T, origin='lower', aspect='auto')
    plot.show()
    
    logspec_tensor = torch.tensor(logspec)
    return logspec_tensor

i = 0
for index in train_csv:
    # 1. load in wavefile
    audio_file_path = "groove/" + index.audio_filename
    wav_file = AudioSegment.from_file(audio_file_path, format="wav")
    # convert sample width if not set to 2 (16 bits)
    if wav_file.sample_width != 2:
        wav_file = wav_file.set_sample_width(2)
        # print("\tnew sample_width: ", wav_file.sample_width)
        wav_file.export(audio_file_path, format="wav")
    # convert file from stereo to mono if channels > 1
    if wav_file.channels != 1:
        wav_file = wav_file.set_channels(1)
        wav_file.export(audio_file_path, format="wav")

    print ("frame rate: ", wav_file.frame_rate)
    tensor = load_audio_to_melspec_tensor(audio_file_path, wav_file.frame_rate)
    
    print (i, "\tfile: ", index.audio_filename, "\tduration: ", wav_file.duration_seconds, "\tensor: ", tensor.shape)

    i += 1